In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00


In [ ]:
import os, copy
import pandas as pd
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import warnings
from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/Drive/", force_remount=True)


Mounted at /content/Drive/


In [ ]:
origin_df = pd.read_csv('/content/Drive/MyDrive/NP_classifier/mod_intent_model_dataset.csv')

In [ ]:
df = copy.deepcopy(origin_df)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.dropna(inplace=True)
df.drop(df[df.duplicated()].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 부정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(표현) 긍정감정 표현하기'].index, axis=0, inplace=True)
df.drop(df[df['intent'] == '(선언/위임하기)'].index, axis=0, inplace=True)
df.drop(df.loc[df['sentence'].str.contains('\*')].index, axis=0, inplace=True)
df = df.loc[df['sentence'].str.len() <= 100]
df.reset_index(drop=True, inplace=True)
df

,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,(단언) 진술하기
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,(지시) 충고/제안하기
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,(단언) 진술하기
3,아닌데 그냥 많이 먹은 거 같은데,(단언) 주장하기
4,그냥 나 뭐 사주기 싫어서 그런 거 같은데.,(단언) 주장하기
...,...,...
1129971,안녕하세유,(표현) 인사하기
1129972,매물 위치와 평수를 알 수 있을까요?,(지시) 질문하기
1129973,잘 진행되고 있나요?,(지시) 질문하기
1129974,네~ 안녕하세요,(표현) 인사하기


In [ ]:
# 클래스 개수
lable_num = len(df['intent'].unique())
lable_num

13

In [ ]:
# str -> int 라벨변경
intet_label = list(df['intent'].unique())

label_dict = {}

for idx, intent_lab in enumerate(intet_label) :
    df.loc[df['intent'] == intent_lab, 'intent'] = idx
    label_dict[idx] = intent_lab

print(label_dict)
df

{0: '(단언) 진술하기', 1: '(지시) 충고/제안하기', 2: '(단언) 주장하기', 3: '(지시) 질문하기', 4: '(지시) 부탁하기', 5: '(단언) 반박하기', 6: '(표현) 감사하기', 7: '(표현) 사과하기', 8: '(지시) 명령/요구하기', 9: '턴토크 사인(관습적 반응)', 10: '(언약) 약속하기(제3자와)/(개인적 수준)', 11: '(언약) 거절하기', 12: '(표현) 인사하기'}


,sentence,intent
0,애덜앙 나 너무 배불러서 배 아파,0
1,배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,0
3,아닌데 그냥 많이 먹은 거 같은데,2
4,그냥 나 뭐 사주기 싫어서 그런 거 같은데.,2
...,...,...
1129971,안녕하세유,12
1129972,매물 위치와 평수를 알 수 있을까요?,3
1129973,잘 진행되고 있나요?,3
1129974,네~ 안녕하세요,12


In [ ]:
train_data = pd.DataFrame()
test_data = pd.DataFrame()

for k,v in label_dict.items():
    len_ = df.loc[df['intent'] == k,'intent'].count()
    num_ = 100000
    if len_ > num_ :
        temp = df.loc[df['intent'] == k].sample(n=num_).reset_index(drop=True)
        train_data = pd.concat([train_data,temp.loc[:((num_-1)*8)/10]],ignore_index=True)
        test_data = pd.concat([test_data,temp.loc[((num_-1)*8)/10:]],ignore_index=True)
    else :
        temp = df.loc[df['intent'] == k].reset_index(drop=True)
        train_data = pd.concat([train_data,temp.loc[:(len_*8)/10]],ignore_index=True)
        test_data = pd.concat([test_data,temp.loc[(len_*8)/10:]],ignore_index=True)


train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = max(df['sentence'].apply(lambda x: len(str(x))))

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # input_id는 워드임베딩을 위한 문장의 정수인코딩
        input_id = tokenizer.encode(example, max_length=max_seq_len, pad_to_max_length=True, truncation=True)

        # attention_mask는 실제단어가 위치하면 1, 패딩의 위치에는 0인 시퀀스.
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count

        # token_type_id은 세그먼트 인코딩
        token_type_id = [0] * max_seq_len

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention masklength {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length{} vs {}".format(len(token_type_id), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
train_X, train_y = convert_examples_to_features(train_data['sentence'], train_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 294755/294755 [00:53<00:00, 5542.64it/s]


In [ ]:
test_X, test_y = convert_examples_to_features(test_data['sentence'], test_data['intent'],
                                              max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 73684/73684 [00:13<00:00, 5659.46it/s]


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=lable_num, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_X, train_y, epochs=4, batch_size=256, validation_split=0.2)

Epoch 1/4
922/922 [==============================] - 517s 506ms/step - loss: 0.7581 - accuracy: 0.7236 - val_loss: 0.6775 - val_accuracy: 0.7435
Epoch 2/4
922/922 [==============================] - 459s 497ms/step - loss: 0.6452 - accuracy: 0.7602 - val_loss: 0.6705 - val_accuracy: 0.7473
Epoch 3/4
922/922 [==============================] - 458s 497ms/step - loss: 0.5757 - accuracy: 0.7866 - val_loss: 0.6986 - val_accuracy: 0.7420
Epoch 4/4
922/922 [==============================] - 458s 497ms/step - loss: 0.4911 - accuracy: 0.8211 - val_loss: 0.7587 - val_accuracy: 0.7348


In [ ]:
results = model.evaluate(test_X, test_y, batch_size=64)
print("test loss, test acc: ", results)

1152/1152 [==============================] - 61s 53ms/step - loss: 0.7661 - accuracy: 0.7326
test loss, test acc:  [0.7661464810371399, 0.7325742244720459]


In [ ]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])

    print('sentence :',new_sentence)
    print(label_dict[score])


In [ ]:
MODEL_SAVE_PATH = os.path.join("/content/Drive/MyDrive/intent_classifier/intent_classifier")
print(MODEL_SAVE_PATH)

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/Drive/MyDrive/intent_classifier/intent_classifier
/content/Drive/MyDrive/intent_classifier/intent_classifier -- Folder already exists 



('/content/Drive/MyDrive/intent_classifier/intent_classifier/tokenizer_config.json',
 '/content/Drive/MyDrive/intent_classifier/intent_classifier/special_tokens_map.json',
 '/content/Drive/MyDrive/intent_classifier/intent_classifier/vocab.txt',
 '/content/Drive/MyDrive/intent_classifier/intent_classifier/added_tokens.json')

In [ ]:
new_sentence = input('sentence > ')
sentiment_predict(new_sentence)

sentence > 우선 저기로 가보셔요
1/1 [==============================] - 0s 42ms/step
sentence : 우선 저기로 가보셔요
(지시) 충고/제안하기
